# Realisation de toutes les differentes simulations avec la méthode POO

## 1. Importation des libraries

In [1]:
from signal import signal
from tokenize import Single
import numpy as np
from scipy.optimize import minimize
from matplotlib import colors as mcolors

%matplotlib qt
import matplotlib.pyplot as plt

from matplotlib.widgets import Slider, Button, RadioButtons,TextBox,CheckButtons
import matplotlib.patches as mpatches
import datetime
import os
import tclab
import time

from xmlrpc.client import Boolean
import numpy as np

from IPython.display import display, clear_output

import package_Class
from package_Class import *

## 2. Simulations

### 2.1 Réponse unitaire de MV à PV

In [ ]:
SIM = Simulation(2000,1,20,True,'OPL_MV_to_PV')
G = Graph(SIM)
SP = Path(SIM,{0:40 , 1000:60 , SIM.TSim: 60})

P = FirstOrder(SIM,0.66,201.33,1,50,SIM.PVInit)

if(SIM.sim == True):
    t = []
    for ti in SIM.t:
        t.append(ti)
        #Signals
        SP.RT(t)
        SIM.MV.append(SP.Signal[-1])
        P.RT(SIM.MV,'EBD')
        SIM.PV.append(P.PV[-1]) # Point Value

# Signaux Afficher dans  Graph Binaires
SigValsBin = []

# Signaux Afficher dans  Graph 1 : Temperature
SigVals1 = [
    Signal(SP.Signal,'SP','-r'),
    Signal(SIM.PV,'PV','-b')
]

# Signaux Afficher dans Graph 2 : % de chauffe
SigVals2 = [
    Signal(SIM.MV,'MV','-b')
]

# Signaux enregistrer dans le .txt
SigSave = [
    Signal(SIM.MV,'MV','-b'),
    Signal(SP.Signal,'SP',':m'),
    Signal(SIM.PV,'PV',':m')
]

# Variables affichées sur le graph
varVals = [
    Variable(SIM.TSim,'Temps Sim [s]'),
    Variable(SIM.Ts,'Sampling [s]'),
    Variable(SIM.PVInit,'Pv Init [°C]'),

    Variable(P.Kg,'Process Gain K'),
    Variable(P.T,'Process Time T'),
    Variable(P.T,'Process Theta θ'),

]

G.show([SigVals1,SigVals2,SigSave],SigValsBin,varVals)

### 2.2 Réponse unitaire de DV à PV

In [ ]:
SIM = Simulation(2000,1,20,True,'OPL_DV_to_PV')
G = Graph(SIM)
SP = Path(SIM,{0:40 , 1000:60 , SIM.TSim: 60})

D = FirstOrder(SIM,0.4,152,29,50,SIM.PVInit)

if(SIM.sim == True):
    t = []
    for ti in SIM.t:
        t.append(ti)
        #Signals
        SP.RT(t)
        SIM.MV.append(SP.Signal[-1])
        D.RT(SIM.MV,'EBD')
        SIM.PV.append(D.PV[-1]) # Point Value

# Signaux Afficher dans  Graph Binaires
SigValsBin = [
]

# Signaux Afficher dans  Graph 1 : Temperature
SigVals1 = [
    Signal(SP.Signal,'SP','-r'),
    Signal(SIM.PV,'PV','-b')
]

# Signaux Afficher dans Graph 2 : % de chauffe
SigVals2 = [
    Signal(SIM.MV,'MV','-b'),
    Signal(SP.Signal,'SP','-k')
]

# Signaux enregistrer dans le .txt
SigSave = [
    Signal(SIM.MV,'MV','-b'),
    Signal(SP.Signal,'SP',':m'),
    Signal(SIM.PV,'PV',':m')

]

# Variables affichées sur le graph
varVals = [
    Variable(SIM.TSim,'Temps Sim [s]'),
    Variable(SIM.Ts,'Sampling [s]'),
    Variable(SIM.PVInit,'Pv Init [°C]'),

    Variable(D.Kg,'Disturbance Gain K'),
    Variable(D.T,'Disturbance Time T'),
    Variable(D.T,'Disturbance Theta θ')


]

G.show([SigVals1,SigVals2,SigSave],SigValsBin,varVals)

### 2.3 Réponse a boucle fermé sur un changement de consigne SP

In [ ]:
SIM = Simulation(500,0.1,20,True,'CPL_SP')
G = Graph(SIM)

SP = Path(SIM,{0:40, 250:60 , SIM.TSim: 60})
DV = Path(SIM,{0:50 ,SIM.TSim: 50})

P = FirstOrder(SIM,0.66,201.33,1,50,SIM.PVInit)
D = FirstOrder(SIM,0.4,152,29,50,0)

PID = P_Controller(SIM,10,[0,100],False,False)

if(SIM.sim == True):
    t = []
    for ti in SIM.t:
        t.append(ti)
        #Signals
        SP.RT(t)
        DV.RT(t)

        PID.RT(SP.Signal,SIM.PV,'EBD-EBD')

        SIM.MV.append(PID.MVFB[-1])

        P.RT(SIM.MV,'EBD')
        D.RT(DV.Signal,'EBD')
        
        SIM.PV.append(P.PV[-1]+D.PV[-1])  # Point Value

if(SIM.sim == False):
    #Tc Lab
    LAB = tclab.TCLab()
    LABVal = LabValues(SIM,LAB)

    SIM.t = []
    start = time.time()
    delta = 0
    totalTime = 0
    last = time.time()

    while totalTime < SIM.TSim:
        if delta > SIM.Ts:
            last = time.time()
            SIM.t.append(round(totalTime,4))
            #Signals
            SP.RT(SIM.t)
            DV.RT(SIM.t)

            PID.RT(SP.Signal,SIM.PV,'EBD-EBD')
    
            SIM.MV.append(PID.MVFB[-1])
    
            LABVal.RT(SIM.MV,DV.Signal,D.point_fct)
            delta = 0
            SIM.updateBar()

        else :
            totalTime = time.time() - start
            delta = time.time() - last
    LAB.close()

# Signaux Afficher dans  Graph Binaires
SigValsBin = [
]

# Signaux Afficher dans  Graph 1 : Temperature
SigVals1 = [
    Signal(SP.Signal,'SP','-r'),
    Signal(SIM.PV,'PV','-b')
]

# Signaux Afficher dans Graph 2 : % de chauffe
SigVals2 = [
    Signal(SIM.MV,'MV','-g'),
    Signal(DV.Signal,'DV','-k'),
    Signal(PID.MVFB,'MVFB','-y'),
]

# Signaux enregistrer dans le .txt
SigSave = [
    Signal(SIM.MV,'MV','-b'),
    Signal(PID.MVP,'MVP','-b'),
    Signal(SP.Signal,'SP',':m'),
    Signal(SIM.PV,'PV',':m'),
    Signal(DV.Signal,'DV',':m')

]

# Variables affichées sur le graph
varVals = [
    Variable(SIM.TSim,'Temps Sim [s]'),
    Variable(SIM.Ts,'Sampling [s]'),
    Variable(SIM.PVInit,'Pv Init [°C]'),

    Variable(PID.Kc,'FeedBack Gain'),


    Variable(P.Kg,'P(s) Gain K'),
    Variable(P.T,'P(s) Time T'),
    Variable(P.T,'P(s) Theta θ'),

    Variable(D.Kg,'D(s) Gain K'),
    Variable(D.T,'D(s) Time T'),
    Variable(D.T,'D(s) Theta θ')
]

G.show([SigVals1,SigVals2,SigSave],SigValsBin,varVals)

### 2.4 Réponse a DV : Pas de FF et controleur en mode manuelle

In [ ]:
SIM = Simulation(700,0.1,20,True,'CPL_NO_FF_MAN_MODE')
G = Graph(SIM)

SP = Path(SIM,{0:40, 250:60 , SIM.TSim: 60})
DV = Path(SIM,{0:50 ,SIM.TSim: 50})
MAN = Path(SIM,{0:1 ,50: 0,500: 1, SIM.TSim: 0})
MANV = Path(SIM,{0: 70,400:100,600:30, SIM.TSim: 100})

P = FirstOrder(SIM,0.66,201.33,1,50,SIM.PVInit)
D = FirstOrder(SIM,0.4,152,29,50,0)

PID = PID_Controller(SIM,5,200,0,0.14,[0,100],False,False)

if(SIM.sim == True):
    t = []
    for ti in SIM.t:
        t.append(ti)
        #Signals
        SP.RT(t)
        DV.RT(t)
        MAN.RT(t)
        MANV.RT(t)

        PID.RT(SP.Signal,SIM.PV,MAN.Signal,MANV.Signal,[0],'EBD-EBD')

        SIM.MV.append(PID.MVFB[-1])


        P.RT(SIM.MV,'EBD')
        D.RT(DV.Signal,'EBD')
        
        SIM.PV.append(P.PV[-1]+D.PV[-1])  # Point Value

if(SIM.sim == False):
    #Tc Lab
    LAB = tclab.TCLab()
    LABVal = LabValues(SIM,LAB)

    SIM.t = []
    start = time.time()
    delta = 0
    totalTime = 0
    last = time.time()

    while totalTime < SIM.TSim:
        if delta > SIM.Ts:
            last = time.time()
            SIM.t.append(round(totalTime,4))
            #Signals
            SP.RT(SIM.t)
            DV.RT(SIM.t)
            MAN.RT(SIM.t)
            MANV.RT(SIM.t)

            PID.RT(SP.Signal,SIM.PV,MAN.Signal,MANV.Signal,[0],'EBD-EBD')
    
            SIM.MV.append(PID.MVFB[-1])

            LABVal.RT(SIM.MV,DV.Signal,D.point_fct)
            delta = 0
            SIM.updateBar()

        else :
            totalTime = time.time() - start
            delta = time.time() - last
    LAB.close()

# Signaux Afficher dans  Graph Binaires
SigValsBin = [
    Signal(MAN.Signal,'Manual Mode','-g'),
]

# Signaux Afficher dans  Graph 1 : Temperature
SigVals1 = [
    Signal(SP.Signal,'SP','-r'),
    Signal(SIM.PV,'PV','-b')
]

# Signaux Afficher dans Graph 2 : % de chauffe
SigVals2 = [
    Signal(SIM.MV,'MV','-g'),
    Signal(DV.Signal,'DV','-k'),
    Signal(PID.MVFB,'MVFB','-y'),
    Signal(PID.MVP,'MVP',':g'),
    Signal(PID.MVI,'MVI',':y'),
    Signal(PID.MVD,'MVD',':m'),
]

# Signaux enregistrer dans le .txt
SigSave = [
    Signal(SIM.MV,'MV','-b'),
    Signal(PID.MVP,'MVP','-b'),
    Signal(PID.MVI,'MVI','-b'),
    Signal(PID.MVD,'MVD','-b'),
    Signal(SP.Signal,'SP',':m'),
    Signal(SIM.PV,'PV',':m'),
    Signal(DV.Signal,'DV',':m'),

]

# Variables affichées sur le graph
varVals = [
    Variable(SIM.TSim,'Temps Sim [s]'),
    Variable(SIM.Ts,'Sampling [s]'),
    Variable(SIM.PVInit,'Pv Init [°C]'),

    Variable(PID.OLP,'Open Loop'),
    Variable(PID.ManFF,'Man FF'),

    Variable(PID.Kc,'Kc PID'),
    Variable(PID.Td,'Td PID'),
    Variable(PID.Ti,'Ti PID'),

    Variable(P.Kg,'P(s) Gain K'),
    Variable(P.T,'P(s) Time T'),
    Variable(P.T,'P(s) Theta θ'),

    Variable(D.Kg,'D(s) Gain K'),
    Variable(D.T,'D(s) Time T'),
    Variable(D.T,'D(s) Theta θ'),
]

G.show([SigVals1,SigVals2,SigSave],SigValsBin,varVals)

### 2.5 Réponse a DV : FF et controleur en mode manuelle

In [ ]:
SIM = Simulation(700,0.1,20,True,'CPL_NO_FF_MAN_MODE')
G = Graph(SIM)

SP = Path(SIM,{0:40, 250:60 , SIM.TSim: 60})
DV = Path(SIM,{0:50 ,SIM.TSim: 50})
MAN = Path(SIM,{0:1 ,50: 0,500: 1, SIM.TSim: 0})
MANV = Path(SIM,{0: 70,400:100,600:30, SIM.TSim: 100})

P = FirstOrder(SIM,0.66,201.33,1,50,SIM.PVInit)
D = FirstOrder(SIM,0.4,152,29,50,0)

PID = PID_Controller(SIM,5,200,0,0.14,[0,100],False,False)

if(SIM.sim == True):
    t = []
    for ti in SIM.t:
        t.append(ti)
        #Signals
        SP.RT(t)
        DV.RT(t)
        MAN.RT(t)
        MANV.RT(t)

        PID.RT(SP.Signal,SIM.PV,MAN.Signal,MANV.Signal,[0],'EBD-EBD')

        SIM.MV.append(PID.MVFB[-1])


        P.RT(SIM.MV,'EBD')
        D.RT(DV.Signal,'EBD')
        
        SIM.PV.append(P.PV[-1]+D.PV[-1])  # Point Value

if(SIM.sim == False):
    #Tc Lab
    LAB = tclab.TCLab()
    LABVal = LabValues(SIM,LAB)

    SIM.t = []
    start = time.time()
    delta = 0
    totalTime = 0
    last = time.time()

    while totalTime < SIM.TSim:
        if delta > SIM.Ts:
            last = time.time()
            SIM.t.append(round(totalTime,4))
            #Signals
            SP.RT(SIM.t)
            DV.RT(SIM.t)
            MAN.RT(SIM.t)
            MANV.RT(SIM.t)

            PID.RT(SP.Signal,SIM.PV,MAN.Signal,MANV.Signal,[0],'EBD-EBD')
    
            SIM.MV.append(PID.MVFB[-1])

            LABVal.RT(SIM.MV,DV.Signal,D.point_fct)
            delta = 0
            SIM.updateBar()

        else :
            totalTime = time.time() - start
            delta = time.time() - last
    LAB.close()

# Signaux Afficher dans  Graph Binaires
SigValsBin = [
    Signal(MAN.Signal,'Manual Mode','-g'),
]

# Signaux Afficher dans  Graph 1 : Temperature
SigVals1 = [
    Signal(SP.Signal,'SP','-r'),
    Signal(SIM.PV,'PV','-b')
]

# Signaux Afficher dans Graph 2 : % de chauffe
SigVals2 = [
    Signal(SIM.MV,'MV','-g'),
    Signal(DV.Signal,'DV','-k'),
    Signal(PID.MVFB,'MVFB','-y'),
    Signal(PID.MVP,'MVP',':g'),
    Signal(PID.MVI,'MVI',':y'),
    Signal(PID.MVD,'MVD',':m'),
]

# Signaux enregistrer dans le .txt
SigSave = [
    Signal(SIM.MV,'MV','-b'),
    Signal(PID.MVP,'MVP','-b'),
    Signal(PID.MVI,'MVI','-b'),
    Signal(PID.MVD,'MVD','-b'),
    Signal(SP.Signal,'SP',':m'),
    Signal(SIM.PV,'PV',':m'),
    Signal(DV.Signal,'DV',':m'),

]

# Variables affichées sur le graph
varVals = [
    Variable(SIM.TSim,'Temps Sim [s]'),
    Variable(SIM.Ts,'Sampling [s]'),
    Variable(SIM.PVInit,'Pv Init [°C]'),

    Variable(PID.OLP,'Open Loop'),
    Variable(PID.ManFF,'Man FF'),

    Variable(PID.Kc,'Kc PID'),
    Variable(PID.Td,'Td PID'),
    Variable(PID.Ti,'Ti PID'),

    Variable(P.Kg,'P(s) Gain K'),
    Variable(P.T,'P(s) Time T'),
    Variable(P.T,'P(s) Theta θ'),

    Variable(D.Kg,'D(s) Gain K'),
    Variable(D.T,'D(s) Time T'),
    Variable(D.T,'D(s) Theta θ'),
]

G.show([SigVals1,SigVals2,SigSave],SigValsBin,varVals)

### 2.6 Recap All test

In [ ]:
SIM = Simulation(2000,1,20,True,'CPL_FF_MAN_MODE_ALL')
G = Graph(SIM)

SP = Path(SIM,{0:40, 250:60 , SIM.TSim: 60})
DV = Path(SIM,{0:50 ,SIM.TSim: 50})
MAN = Path(SIM,{0:1 , 250:0 ,500: 1, SIM.TSim: 0})
MANV = Path(SIM,{0: 70,400:100,600:30, SIM.TSim: 100})

'''
-MV
K: 0.6380496617368366
T1: 194.68042676186926
T2: 4.132530123359063e-11
theta: 5.142082869041576
-DV
K: 0.41729187227800746
T1: 165.26136274440154
T2: 17.712030277318842
theta: 7.447021274687229
'''

P = SecondOrder(SIM,0.64,194.7,4.13e-11,5.14,50,SIM.PVInit)
D = SecondOrder(SIM,0.42,165.26,17.71,7.45,50,0)

FF = FeedForward(SIM,P,D,True)

PID = PID_Controller(SIM,5,100,0,0.14,[0,100],False,False)
PID.IMC_tuning(P,0.2,'B')



if(SIM.sim == True):
    t = []
    for ti in SIM.t:
        t.append(ti)
        #Signals
        SP.RT(t)
        DV.RT(t)
        MAN.RT(t)
        MANV.RT(t)

        PID.RT(SP.Signal,SIM.PV,MAN.Signal,MANV.Signal,[0],'EBD-EBD')

        SIM.MV.append(PID.MVFB[-1])


        P.RT(SIM.MV,'EBD')
        D.RT(DV.Signal,'EBD')
        
        SIM.PV.append(P.PV[-1]+D.PV[-1])  # Point Value

if(SIM.sim == False):
    #Tc Lab
    LAB = tclab.TCLab()
    LABVal = LabValues(SIM,LAB)

    SIM.t = []
    start = time.time()
    delta = 0
    totalTime = 0
    last = time.time()

    while totalTime < SIM.TSim:
        if delta > SIM.Ts:
            last = time.time()
            SIM.t.append(round(totalTime,4))
            #Signals
            SP.RT(SIM.t)
            DV.RT(SIM.t)
            MAN.RT(SIM.t)
            MANV.RT(SIM.t)

            PID.RT(SP.Signal,SIM.PV,MAN.Signal,MANV.Signal,[0],'EBD-EBD')
    
            SIM.MV.append(PID.MVFB[-1])

            LABVal.RT(SIM.MV,DV.Signal,D.point_fct)
            delta = 0
            SIM.updateBar()

        else :
            totalTime = time.time() - start
            delta = time.time() - last
    LAB.close()

# Signaux Afficher dans  Graph Binaires
SigValsBin = [
    Signal(MAN.Signal,'Manual Mode','-g'),
]

# Signaux Afficher dans  Graph 1 : Temperature
SigVals1 = [
    Signal(SP.Signal,'SP','-r'),
    Signal(SIM.PV,'PV','-b')
]

# Signaux Afficher dans Graph 2 : % de chauffe
SigVals2 = [
    Signal(SIM.MV,'MV','-g'),
    Signal(DV.Signal,'DV','-k'),
    Signal(PID.MVFB,'MVFB','-y'),
    Signal(PID.MVP,'MVP',':g'),
    Signal(PID.MVI,'MVI',':y'),
    Signal(PID.MVD,'MVD',':m'),
]

# Signaux enregistrer dans le .txt
SigSave = [
    Signal(SIM.MV,'MV','-b'),
    Signal(PID.MVP,'MVP','-b'),
    Signal(PID.MVI,'MVI','-b'),
    Signal(PID.MVD,'MVD','-b'),
    Signal(SP.Signal,'SP',':m'),
    Signal(SIM.PV,'PV',':m'),
    Signal(DV.Signal,'DV',':m'),

]

# Variables affichées sur le graph
varVals = [
    Variable(SIM.TSim,'Temps Sim [s]'),
    Variable(SIM.Ts,'Sampling [s]'),
    Variable(SIM.PVInit,'Pv Init [°C]'),

    Variable(PID.OLP,'Open Loop'),
    Variable(PID.ManFF,'Man FF'),

    Variable(PID.Kc,'Kc PID'),
    Variable(PID.Td,'Td PID'),
    Variable(PID.Ti,'Ti PID'),

    Variable(P.Kg,'P(s) Gain K'),
    Variable(P.T,'P(s) Time T'),
    Variable(P.T,'P(s) Theta θ'),

    Variable(D.Kg,'D(s) Gain K'),
    Variable(D.T,'D(s) Time T'),
    Variable(D.T,'D(s) Theta θ'),
]

G.show([SigVals1,SigVals2,SigSave],SigValsBin,varVals)